In [1]:
import pandas as pd
import numpy as np
import time as TIME
import datetime as dt
from datetime import date
from datetime import timedelta
import math
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel('fno_stocks.xlsx', header= 1)
Tickers = (df["Symbol"])
Tickers = Tickers.astype(str)

Tickers = Tickers.drop(Tickers[Tickers.values == "BANKNIFTY"].index.values[0], axis =0)
Tickers = Tickers.drop(Tickers[Tickers.values == "NIFTY"].index.values[0], axis =0)
Tickers = Tickers.reset_index(drop = True)
Tickers

0            IDEA
1      COROMANDEL
2            ZEEL
3       NESTLEIND
4        ASHOKLEY
          ...    
167    APOLLOHOSP
168        INDIGO
169    CANFINHOME
170    ICICIPRULI
171       SBILIFE
Name: Symbol, Length: 172, dtype: object

In [3]:
total_data = pd.read_csv("Day_Wise_Stats_Combined.csv", index_col = 0, parse_dates = [2])
total_data

,AD_Ratio,Date,Stock1,Stock2,Stock3,Stock4,Stock5
0,0.951220,2017-01-02 00:00:00+05:30,BHARATFORG,ASTRAL,MOTHERSUMI,PETRONET,BHEL
1,1.461538,2017-01-03 00:00:00+05:30,IOC,SUNTV,DEEPAKNTR,PNB,ESCORTS
2,1.222222,2017-01-04 00:00:00+05:30,NMDC,COFORGE,IPCALAB,INDUSTOWER,M&MFIN
3,0.975309,2017-01-05 00:00:00+05:30,JSWSTEEL,COROMANDEL,IOC,ADANIPORTS,WIPRO
4,0.860465,2017-01-06 00:00:00+05:30,PFC,APOLLOTYRE,DLF,CANBK,CANFINHOME
...,...,...,...,...,...,...,...
1202,0.722772,2021-09-17 09:15:00+05:30,POLYCAB,INDIGO,JUBLFOOD,BAJAJFINSV,MCDOWELL-N
1203,5.692308,2021-09-20 09:15:00+05:30,INDHOTEL,GMRINFRA,PVR,ZEEL,ASHOKLEY
1204,0.251799,2021-09-21 09:15:00+05:30,ONGC,GODREJPROP,GRASIM,JSWSTEEL,PVR
1205,1.597015,2021-09-22 09:15:00+05:30,ZEEL,INDHOTEL,BALKRISIND,APOLLOTYRE,TECHM


In [12]:
%%time
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
Daily_pct_change = pd.DataFrame()
current_date = date(2017,1,1)
todays_date  = date(2021,9,23)
current_date = pd.Timestamp(current_date, tz =19800)
flag = 0
while(current_date.date() <= todays_date):
    
    start = dt.datetime.now()
    pct_change = {}
    Daily_pct_change = pd.read_csv("Short_Selling_2.csv", index_col = 0,  parse_dates = [2])
    if(flag==0):
        current_date = Daily_pct_change.Date.iloc[-1] + timedelta(days =1)
        flag = 1
        if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
            print(str(current_date) + " : Started")
            print(str(current_date) + " : Finished") 
            current_date = current_date + timedelta(days =1)
            if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
                print(str(current_date) + " : Started")
                print(str(current_date) + " : Finished") 
                current_date = current_date + timedelta(days =1)    
    
    if(total_data[total_data.Date == current_date].AD_Ratio.iloc[-1] > 0.8):
        print(str(current_date) + " : AD Greater than 0.8 ")
        index = total_data[total_data.Date == current_date].index[0]
        current_date = total_data.Date.iloc[index + 1]
        continue
        
    print(str(current_date) + " : Started")
    for ticker in Tickers:
        daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [1])
        daily_data = daily_data[daily_data.date <= pd.Timestamp(current_date) + timedelta(days =1)]
        daily_data = daily_data[daily_data.date >= pd.Timestamp(current_date)]
        daily_data = daily_data.reset_index(drop = True)
        if(daily_data.empty == True):
            continue
        change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
        pct_change[ticker] = change_value    
    
    top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
    change = pd.DataFrame()
    change["Tickers"] = top_gainers.keys()
    change["PCT_Change"] = top_gainers.values()
    
    if(change.empty == True):
        print(str(current_date) + " : Finished") 
        continue
    gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
    loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
    ad_ratio    = gain_number/loss_number
    end = dt.datetime.now() - start
    print("Time Required to calculate pct change: " + str(end)) 
    print("PCT Change Calculated!")
    for i in range(5):
        print(change["Tickers"].iloc[-(i+1)])

    new_row = {"Date": current_date, "Stock1": change["Tickers"].iloc[-1], "Stock2" : change["Tickers"].iloc[-2], "Stock3": change["Tickers"].iloc[-3], "Stock4": change["Tickers"].iloc[-4],"Stock5": change["Tickers"].iloc[-5] , "AD_Ratio":ad_ratio }
    Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
    Daily_pct_change.to_csv("Short_Selling_2.csv")
    
    print(str(current_date) + " : Finished")
    index = total_data[total_data.Date == current_date].index[0]
    current_date = total_data.Date.iloc[index + 1]

2020-03-25 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:48.789027
PCT Change Calculated!
INDIAMART
APOLLOHOSP
ITC
IEX
HDFCAMC
2020-03-25 09:15:00+05:30 : Finished
2020-03-26 09:15:00+05:30 : AD Greater than 0.8 
2020-03-27 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:33.800137
PCT Change Calculated!
CUB
NAM-INDIA
ASTRAL
CANFINHOME
ICICIBANK
2020-03-27 09:15:00+05:30 : Finished
2020-03-30 09:15:00+05:30 : AD Greater than 0.8 
2020-03-31 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:31.977700
PCT Change Calculated!
BANDHANBNK
RBLBANK
AUBANK
CANFINHOME
COROMANDEL
2020-03-31 09:15:00+05:30 : Finished
2020-04-01 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:33.370555
PCT Change Calculated!
PVR
KOTAKBANK
BRITANNIA
BPCL
LT
2020-04-01 09:15:00+05:30 : Finished
2020-04-03 09:15:00+05:30 : AD Greater than 0.8 
2020-04-06 09:15:00+05:30 : AD Greater than 0.8 
2020-04-07 09:15:00+05:30 : AD Greater than 0

KeyboardInterrupt: 

In [10]:
daily_data[:10]

,date,open,high,low,close,volume,oi
0,2020-03-23 09:15:00+05:30,3.25,3.35,3.15,3.30,21347848,0
1,2020-03-23 09:20:00+05:30,3.25,3.30,3.20,3.25,13870813,0
2,2020-03-23 09:25:00+05:30,3.25,3.30,3.25,3.25,9699183,0
3,2020-03-23 09:30:00+05:30,3.25,3.40,3.25,3.35,9942579,0
4,2020-03-23 09:35:00+05:30,3.35,3.35,3.25,3.25,6297059,0
5,2020-03-23 09:40:00+05:30,3.25,3.35,3.25,3.30,4921759,0
6,2020-03-23 09:45:00+05:30,3.30,3.30,3.25,3.30,2305556,0
7,2020-03-23 09:50:00+05:30,3.30,3.30,3.20,3.25,16431303,0
8,2020-03-23 09:55:00+05:30,3.25,3.25,3.20,3.20,1172503,0
9,2020-03-23 10:55:00+05:30,3.25,3.35,3.25,3.25,4957229,0


In [4]:
current_date = date(2019,1,1)
current_date = pd.Timestamp(current_date, tz =19800)
Daily_pct_change = pd.DataFrame()
#todays_date = dt.datetime.now().date()
todays_date  = date(2021,9,23)
pct_change = {}
start = dt.datetime.now()
print(str(current_date) + " : Started")
for ticker in Tickers:
    daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [1])
    daily_data = daily_data[daily_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
    daily_data = daily_data[daily_data.date >= pd.Timestamp(current_date)]
    daily_data = daily_data.reset_index(drop = True)
    if(daily_data.empty == True):
        continue
    change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
    pct_change[ticker] = change_value
    #rint(change_value)

top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
change = pd.DataFrame()
change["Tickers"] = top_gainers.keys()
change["PCT_Change"] = top_gainers.values()
gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
ad_ratio    = gain_number/loss_number
end = dt.datetime.now() - start
print("Time Required to calculate pct change: " + str(end)) 
print("PCT Change Calculated!")
for i in range(5):
    print(change["Tickers"].iloc[-(i+1)])
    
new_row = {"Date": current_date, "Stock1": change["Tickers"].iloc[-1], "Stock2" : change["Tickers"].iloc[-2], "Stock3": change["Tickers"].iloc[-3], "Stock4": change["Tickers"].iloc[-4],"Stock5": change["Tickers"].iloc[-5] , "AD_Ratio":ad_ratio }
Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
Daily_pct_change.to_csv("Short_Selling_2.csv")

date_data = pd.read_csv("Historical_Data/" + change["Tickers"][0] + ".csv", index_col = 0, parse_dates = [1])
date_data = date_data[date_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
date_data = date_data[date_data.date >= pd.Timestamp(current_date)]
date_data = date_data.reset_index(drop = True)
current_date = date_data.date.iloc[-1]
print(str(current_date) + " : Finished")


2019-01-01 00:00:00+05:30 : Started
Time Required to calculate pct change: 0:03:32.880939
PCT Change Calculated!
SRF
IDEA
NMDC
INDUSINDBK
ZEEL
2019-01-02 09:15:00+05:30 : Finished
